In [68]:
import datetime
import os
import requests
import pandas as pd
import importlib
import zipfile
from tqdm import tqdm_notebook as tqdm
import glob
from bs4 import BeautifulSoup
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser

In [69]:
df202304 = pd.read_csv('202304.csv')
df202304

,期間,株式コード,上場コード,銘柄名称,取引所
0,2023/04,1301,11.0,極洋,1.0
1,2023/04,1305,11.0,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算,1.0
2,2023/04,1306,11.0,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上,1.0
3,2023/04,1307,NaN,ＩシェアーズＴＯＰＩＸ,NaN
4,2023/04,1308,11.0,上場インデックスファンドＴＯＰＩＸ,1.0
...,...,...,...,...,...
6857,2023/04,9993,12.0,ヤマザワ,1.0
6858,2023/04,9994,12.0,やまや,1.0
6859,2023/04,9995,11.0,グローセル,1.0
6860,2023/04,9996,12.0,サトー商会,1.0


In [70]:
df202304 = df202304.drop('期間', axis=1)
df202304 = df202304.drop('上場コード', axis=1)
df202304 = df202304.drop('取引所', axis=1)
df202304 = df202304.rename(columns={
    '株式コード':'code',
    '銘柄名称':'name'
})
df202304['code'] = df202304['code'] * 10

In [71]:
df202304

,code,name
0,13010,極洋
1,13050,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上
3,13070,ＩシェアーズＴＯＰＩＸ
4,13080,上場インデックスファンドＴＯＰＩＸ
...,...,...
6857,99930,ヤマザワ
6858,99940,やまや
6859,99950,グローセル
6860,99960,サトー商会


In [72]:
def specify_time_period():
    start_date = datetime.date(2023, 3, 31)
    end_date = datetime.date(2023, 6, 30)
    #決算後3ヶ月以内に有報を提出しなければならない規則から、end_dateを2022/06/30に指定

    period = end_date - start_date
    period = int(period.days)
    day_list = []
    for d in range(period):
        day = start_date + datetime.timedelta(days=d)
        day_list.append(day)
        
    day_list.append(end_date)

    return day_list

In [73]:
def get_report_list(day_list, codes):
    #結果を格納するための空のリストを用意
    report_list =[]
    #日付リストの期間に提出された書類のメタデータを取得してjson形式に変換
    for day in day_list:
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        print(day)
        res = requests.get(url, params=params)
        print(res)
        if res.status_code == 200:
            try:
                json_data = res.json()
            except ValueError:
                print("Invalid JSON format")
        else:
            print("Request failed with status code:", res.status_code)
        for num in range(len(json_data["results"])):
            ordinance_code = json_data["results"][num]["ordinanceCode"]
            form_code = json_data["results"][num]["formCode"]
            sec_code = json_data["results"][num]["secCode"]
            
        #ordinance_code=010かつform_code=030000が有価証券報告書
            if ordinance_code == "010" and form_code =="030000"and sec_code in codes:
                company_name=json_data["results"][num]["filerName"]
                edi={ '会社名':company_name,
                            '書類名':json_data["results"][num]["docDescription"],
                            'docID':json_data["results"][num]["docID"],
                            '証券コード':json_data["results"][num]["secCode"],
                            '開始時期':json_data["results"][num]["periodStart"],
                            '終了時期':json_data["results"][num]["periodEnd"],
                            '日付': day,
                            'XBRL':json_data["results"][num]["xbrlFlag"]}
                report_list.append(edi)

    return report_list

In [74]:
day_list = specify_time_period()
int_codes = df202304['code'].tolist()
codes = [str(num) for num in int_codes]
report_list = get_report_list(day_list, codes)


2023-03-31
<Response [200]>
2023-04-01
<Response [200]>
2023-04-02
<Response [200]>
2023-04-03
<Response [200]>
2023-04-04
<Response [200]>
2023-04-05
<Response [200]>
2023-04-06
<Response [200]>
2023-04-07
<Response [200]>
2023-04-08
<Response [200]>
2023-04-09
<Response [200]>
2023-04-10
<Response [200]>
2023-04-11
<Response [200]>
2023-04-12
<Response [200]>
2023-04-13
<Response [200]>
2023-04-14
<Response [200]>
2023-04-15
<Response [200]>
2023-04-16
<Response [200]>
2023-04-17
<Response [200]>
2023-04-18
<Response [200]>
2023-04-19
<Response [200]>
2023-04-20
<Response [200]>
2023-04-21
<Response [200]>
2023-04-22
<Response [200]>
2023-04-23
<Response [200]>
2023-04-24
<Response [200]>
2023-04-25
<Response [200]>
2023-04-26
<Response [200]>
2023-04-27
<Response [200]>
2023-04-28
<Response [200]>
2023-04-29
<Response [200]>
2023-04-30
<Response [200]>
2023-05-01
<Response [200]>
2023-05-02
<Response [200]>
2023-05-03
<Response [200]>
2023-05-04
<Response [200]>
2023-05-05
<Response

In [75]:
print(report_list)

[{'会社名': '株式会社タダノ', '書類名': '有価証券報告書－第75期(2022/04/01－2022/12/31)', 'docID': 'S100QFN2', '証券コード': '63950', '開始時期': '2022-04-01', '終了時期': '2022-12-31', '日付': datetime.date(2023, 3, 31), 'XBRL': '1'}, {'会社名': '株式会社ナカニシ', '書類名': '有価証券報告書－第71期(2022/01/01－2022/12/31)', 'docID': 'S100QHRO', '証券コード': '77160', '開始時期': '2022-01-01', '終了時期': '2022-12-31', '日付': datetime.date(2023, 3, 31), 'XBRL': '1'}, {'会社名': 'ダイトロン株式会社', '書類名': '有価証券報告書－第71期(2022/01/01－2022/12/31)', 'docID': 'S100QH2M', '証券コード': '76090', '開始時期': '2022-01-01', '終了時期': '2022-12-31', '日付': datetime.date(2023, 3, 31), 'XBRL': '1'}, {'会社名': '株式会社日本抵抗器製作所', '書類名': '有価証券報告書－第75期(2022/01/01－2022/12/31)', 'docID': 'S100QHLH', '証券コード': '69770', '開始時期': '2022-01-01', '終了時期': '2022-12-31', '日付': datetime.date(2023, 3, 31), 'XBRL': '1'}, {'会社名': '湖北工業株式会社', '書類名': '有価証券報告書－第64期(2022/01/01－2022/12/31)', 'docID': 'S100QIMK', '証券コード': '65240', '開始時期': '2022-01-01', '終了時期': '2022-12-31', '日付': datetime.date(2023, 3, 31), 'XBRL': '1'}, {'会社名': '株

In [76]:
df = pd.DataFrame(report_list)
df

,会社名,書類名,docID,証券コード,開始時期,終了時期,日付,XBRL
0,株式会社タダノ,有価証券報告書－第75期(2022/04/01－2022/12/31),S100QFN2,63950,2022-04-01,2022-12-31,2023-03-31,1
1,株式会社ナカニシ,有価証券報告書－第71期(2022/01/01－2022/12/31),S100QHRO,77160,2022-01-01,2022-12-31,2023-03-31,1
2,ダイトロン株式会社,有価証券報告書－第71期(2022/01/01－2022/12/31),S100QH2M,76090,2022-01-01,2022-12-31,2023-03-31,1
3,株式会社日本抵抗器製作所,有価証券報告書－第75期(2022/01/01－2022/12/31),S100QHLH,69770,2022-01-01,2022-12-31,2023-03-31,1
4,湖北工業株式会社,有価証券報告書－第64期(2022/01/01－2022/12/31),S100QIMK,65240,2022-01-01,2022-12-31,2023-03-31,1
...,...,...,...,...,...,...,...,...
2550,東洋建設株式会社,有価証券報告書－第103期(2022/04/01－2023/03/31),S100RB2H,18900,2022-04-01,2023-03-31,2023-06-30,1
2551,株式会社ヤシマキザイ,有価証券報告書－第79期(2022/04/01－2023/03/31),S100RB26,76770,2022-04-01,2023-03-31,2023-06-30,1
2552,ＲＩＺＡＰグループ株式会社,有価証券報告書－第20期(2022/04/01－2023/03/31),S100RB27,29280,2022-04-01,2023-03-31,2023-06-30,1
2553,株式会社エムケイシステム,有価証券報告書－第35期(2022/04/01－2023/03/31),S100R9Q3,39100,2022-04-01,2023-03-31,2023-06-30,1


In [77]:
def download_zip(docid_list, save_dir):
    filename_list = []


    for docid in tqdm(docid_list):
        # 書類取得APIのエンドポイント
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + docid
        #print(url)
        
        # 書類取得APIのリクエストパラメータ
        params = {
            "type" : 1}
            
        # 出力ファイル名
        filename = docid + ".zip"
        filename_list.append(filename)

        # 出力ファイルパス
        save_path = os.path.join(save_dir, filename)

        # 書類取得APIの呼び出し
        res = requests.get(url, params=params, verify=False)
        # ファイルへ出力
        # print(res.status_code)
        if res.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in res.iter_content(chunk_size=1024):
                    f.write(chunk)

    return filename_list

In [78]:
def extract_zip(docid_list, save_dir, filename_list):
    extract_dir = 'content2/'

    for i, docid in enumerate(docid_list):
        # 出力ファイルパス
        file_path = os.path.join(save_dir, filename_list[i])

        print(file_path)
        
        if zipfile.is_zipfile(file_path) == True:
            with zipfile.ZipFile(file_path) as zip_f:
                # 解凍先ディレクトリを指定して解凍
                extract_path = os.path.join(extract_dir, docid)
                os.makedirs(extract_path, exist_ok=True)
                zip_f.extractall(extract_path)
        else:  # エラー回避
            print(file_path + " is not a Zip File")
            docid_list.remove(docid)

In [79]:
docid_list = df['docID'].tolist()
save_dir = 'zip2/'
filename_list = download_zip(docid_list, save_dir)



<ipython-input-77-29ab0780bfcd>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for docid in tqdm(docid_list):


  0%|          | 0/2555 [00:00<?, ?it/s]

/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


In [80]:
extract_zip(docid_list, save_dir, filename_list)

zip2/S100QFN2.zip
zip2/S100QHRO.zip
zip2/S100QH2M.zip
zip2/S100QHLH.zip
zip2/S100QIMK.zip
zip2/S100QHYF.zip
zip2/S100QIAD.zip
zip2/S100QIFF.zip
zip2/S100QI36.zip
zip2/S100QHBU.zip
zip2/S100QIOK.zip
zip2/S100QIAK.zip
zip2/S100QIGA.zip
zip2/S100QINP.zip
zip2/S100QFJP.zip
zip2/S100QIHL.zip
zip2/S100QHTY.zip
zip2/S100QILX.zip
zip2/S100QIG8.zip
zip2/S100QGXW.zip
zip2/S100QHPQ.zip
zip2/S100QIPI.zip
zip2/S100QBIJ.zip
zip2/S100QHO4.zip
zip2/S100QIQB.zip
zip2/S100QIGI.zip
zip2/S100QIFE.zip
zip2/S100QIMC.zip
zip2/S100QIC1.zip
zip2/S100QIQS.zip
zip2/S100QI2H.zip
zip2/S100QIS7.zip
zip2/S100QIRS.zip
zip2/S100QIRK.zip
zip2/S100QIQN.zip
zip2/S100QH0H.zip
zip2/S100QI89.zip
zip2/S100QIL1.zip
zip2/S100QIT2.zip
zip2/S100QIFG.zip
zip2/S100QIT0.zip
zip2/S100QIBO.zip
zip2/S100QIME.zip
zip2/S100QISL.zip
zip2/S100QIDJ.zip
zip2/S100QFXZ.zip
zip2/S100QI8D.zip
zip2/S100QIUK.zip
zip2/S100QIUQ.zip
zip2/S100QIUW.zip
zip2/S100QH1J.zip
zip2/S100QIS6.zip
zip2/S100QGBC.zip
zip2/S100QIWE.zip
zip2/S100QIJS.zip
zip2/S100Q

In [81]:
parser = EdinetXbrlParser()


content = pd.DataFrame(columns=['docID','経営方針'])


for _docID in tqdm(docid_list):
    #xbrlファイルのパスを指定
    xbrl_path = 'content2/' + _docID + '/XBRL/PublicDoc/*.xbrl'
    # print(i)
    # print(xbrl_path)
    # print(glob.glob(xbrl_path))
    xbrl_path = glob.glob(xbrl_path)[0]
    # print(xbrl_path)
    edinet_xbrl_object = parser.parse_file(xbrl_path)

    #タクソノミ要素となるkeyとcontext_refを指定
    #経営方針、経営環境および対処すべき課題等
    key_houshin= 'jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock'

    #context_refを定義
    context_ref_1 = 'FilingDateInstant'
        
    #例外処理
    try:
        data_houshin = edinet_xbrl_object.get_data_by_context_ref(key_houshin, context_ref_1).get_value()
        soup_houshin = BeautifulSoup(data_houshin,'html.parser').get_text(strip=True)
        
        tmp_se = pd.DataFrame([[_docID], [soup_houshin]], index=content.columns).T
        content = content.append(tmp_se)

    except AttributeError as e:
        print(e)

<ipython-input-81-f7d662bd09d3>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _docID in tqdm(docid_list):


  0%|          | 0/2555 [00:00<?, ?it/s]

/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
<ipython-input-81-f7d662bd09d3>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  content = content.append(tmp_se)
/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning.

In [82]:
content

,docID,経営方針
0,S100QFN2,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日...
0,S100QHRO,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100QH2M,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100QHLH,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末におい...
0,S100QIMK,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項については、当連結会計年...
...,...,...
0,S100RB2H,１【経営方針、経営環境及び対処すべき課題等】当社グループの経営方針、経営環境及び対処すべき課...
0,S100RB26,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、本書提出日現在におい...
0,S100RB27,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100R9Q3,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...


In [83]:
df['会社名']

0             株式会社タダノ
1            株式会社ナカニシ
2           ダイトロン株式会社
3        株式会社日本抵抗器製作所
4            湖北工業株式会社
            ...      
2550         東洋建設株式会社
2551       株式会社ヤシマキザイ
2552    ＲＩＺＡＰグループ株式会社
2553     株式会社エムケイシステム
2554         株式会社ジーニー
Name: 会社名, Length: 2555, dtype: object

In [90]:
content['name'] = df['会社名']
content

,docID,経営方針,name
0,S100QFN2,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日...,株式会社タダノ
1,S100QHRO,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,株式会社ナカニシ
2,S100QH2M,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,ダイトロン株式会社
3,S100QHLH,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末におい...,株式会社日本抵抗器製作所
4,S100QIMK,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項については、当連結会計年...,湖北工業株式会社
...,...,...,...
2550,S100RB2H,１【経営方針、経営環境及び対処すべき課題等】当社グループの経営方針、経営環境及び対処すべき課...,東洋建設株式会社
2551,S100RB26,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、本書提出日現在におい...,株式会社ヤシマキザイ
2552,S100RB27,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,ＲＩＺＡＰグループ株式会社
2553,S100R9Q3,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...,株式会社エムケイシステム


In [ ]:
content.to_csv('datasets_for_predicting.csv', index=False)